# 🧠 RAG Engine in Colab (FAISS + Summarization)
This notebook is a safe version of `rag_engine.py` for running on Google Colab.

In [1]:
# ✅ Step 1: Install dependencies (Colab-safe)
!pip install sentence-transformers faiss-cpu transformers pandas tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [7]:
# ✅ Step 3: RAG Engine Setup
import pandas as pd
import numpy as np
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from transformers import pipeline

class RAGTicketEngine:
    def __init__(self, path="tickets.csv"):
        self.data = pd.read_csv(path)
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")
        self.generator = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
        self._build_vector_store()

    def _build_vector_store(self):
        texts = self.data["body"].tolist()
        self.embeddings = self.embedder.encode(texts, show_progress_bar=True)
        dim = len(self.embeddings[0])
        self.index = faiss.IndexFlatL2(dim)
        self.index.add(np.array(self.embeddings).astype("float32"))

    def query(self, query_text):
        q_emb = self.embedder.encode([query_text])[0]
        D, I = self.index.search(np.array([q_emb]).astype("float32"), k=1)
        row = self.data.iloc[I[0][0]]
        context = f"Ticket: {row.body}\nResolution: {row.resolution}"
        prompt = f"Customer asks: {query_text}\n{context}\n\nSummarize this:"
        summary = self.generator(prompt, max_length=130, min_length=30, do_sample=False)[0]["summary_text"]
        return summary.strip()

In [12]:
from google.colab import drive
drive.mount('/content/drive')

# Correct the path to access the file after mounting Google Drive
csv_path = "/content/drive/My Drive/Colab Notebooks/RAG/tickets.csv"
engine = RAGTicketEngine(path=csv_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Device set to use cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
# ✅ Step 4: Run sample queries
queries_df = pd.DataFrame({
    "id": [1, 2, 3],
    "question": [
        "Why does upload fail for large files?",
        "How do I reset my password?",
        "Why is email delivery delayed?"
    ]
})

# Remove this line: engine = RAGTicketEngine()
results = []

for _, row in tqdm(queries_df.iterrows(), total=len(queries_df)):
    qid = row["id"]
    question = row["question"]
    try:
        # Use the existing 'engine' instance
        answer = engine.query(question)
        results.append({"id": qid, "query": question, "summary": answer})
    except Exception as e:
        results.append({"id": qid, "query": question, "summary": f"ERROR: {e}"})

results_df = pd.DataFrame(results)
results_df.to_csv("query_summaries.csv", index=False)
results_df.head()

100%|██████████| 3/3 [00:39<00:00, 13.00s/it]


,id,query,summary
0,1,Why does upload fail for large files?,Customer asks: Why does upload fail for large ...
1,2,How do I reset my password?,Customer asks: How do I reset my password? Tic...
2,3,Why is email delivery delayed?,Customer asks: Why is email delivery delayed? ...


from matplotlib import pyplot as plt
_df_0['id'].plot(kind='hist', bins=20, title='id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('query').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('summary').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['summary'].value_counts()
    for x_label, grp in _df_6.groupby('query')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('query')
_ = plt.ylabel('summary')

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_7['query'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_7, x='id', y='query', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_8['summary'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_8, x='id', y='summary', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [14]:
# ✅ Step 5: Download results
files.download("query_summaries.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>